In [1]:
using Distributions
σ(x) = 1.0/(1.0+exp(-x));
ϕ(x) = -log(1.0/x -1.0);

In [ ]:
# unknowns
n = 3;

# equations
m = 2;

# valid values of the unknowns
xs = rand(n);

# weights of a neural net layer
W  = rand(Uniform(-sqrt(n+1),sqrt(n+1)), (n+1,m));

# real estimation of xs flowing through the layer
ys = map(σ, W'*[1.0;xs]);

In [ ]:
function narrowBounds(boundsA, boundsB)
    lowBounds  = Array{Float64,1}(length(boundsA)+1)
    highBounds = Array{Float64,1}(length(boundsB)+1)
    lowBounds[1]  = 0.0
    highBounds[1] = 1.0
    for i in eachindex(boundsA)[1:end]
        if boundsA[i] < boundsB[i]
            lowBounds[i+1]  = boundsA[i]
            highBounds[i+1] = boundsB[i]
        else
            lowBounds[i+1]  = boundsB[i]
            highBounds[i+1] = boundsA[i]
        end
    end
    (maximum(lowBounds),minimum(highBounds))
end

In [ ]:
function nextInterval(ys, W, B)
    variables, equations = size(W)
end

In [ ]:
function sendOutput(ys, W)
    # number of variables and equations
    variables, equations = size(W)
    
    # xs are the values to be calculated (except x[1] which is 1.0)
    xs = ones(variables)
    # ks are the constant values in the equations
    ks = (map(ϕ,ys).-W[1,:]')[:]
    
    negativeWeight = zeros(ks)
    positiveWeight = zeros(ks)
    for j in 1:equations
        for i in 2:variables
            w = W[i,j]
            w < 0 ? negativeWeight[j] += w : positiveWeight[j] += w
        end
    end
    
    knownValues = zeros(equations)
    for i in 2:variables
        reciprocals = zeros(equations)
        for j in 1:equations
            W[i,j] < 0 ? negativeWeight[j] -= W[i,j] : positiveWeight[j] -= W[i,j]
            reciprocals[j] = 1.0/W[i,j]
        end
        boundsA = (ks .- negativeWeight .- knownValues).*reciprocals
        boundsB = (ks .- positiveWeight .- knownValues).*reciprocals
        lowBound, highBound = narrowBounds(boundsA, boundsB)
        println("xs[$i] ∈ [$lowBound,$highBound]")
        xs[i] = rand(Uniform(lowBound, highBound))
        for j in 1:equations
            knownValues[j] += xs[i]*W[i,j]
        end
    end
    xs
end

In [ ]:
sendOutput(ys, W)

In [ ]:
function sendOutput(ys, W)
    equations = size(W)[2]
    variables  = size(W)[1]
    
    xs = zeros(variables)           # Array{Float64,1}
    ks = (map(ϕ, ys) - W[1,:]')[:]  # Array{Float64,1}
    
    xs[1] = 1.0                     # first entry is always 1.0
    
    negativeWeight = zeros(ks)
    positiveWeight = zeros(ks)
    for i in 1:size(W)[1]
        for j in 1:size(W)[2]
            w = W[i,j]
            w < 0 ? negativeWeight[j] += w : positiveWeight[j] += w
        end
    end
    
    knownValues = zeros(ks)
    for row in 2:variables
        weights = W[row,:]
        for col in eachindex(weights)
            w = weights[col]
            w < 0 ? negativeWeight[col] -= w : positiveWeight[col] -= w
        end
        reciprocals = (1.0./weights')[:]
        boundsA = (ks .- negativeWeight .- knownValues).*reciprocals
        boundsB = (ks .- positiveWeight .- knownValues).*reciprocals
        lowBound, highBound = narrowBounds(boundsA, boundsB)
        println(lowBound)
        println(highBound)
        println()
        xs[row] = rand(Uniform(lowBound, highBound))
        knownValues = knownValues .+ (xs[row] .* weights)
    end
    xs
end

In [ ]:
y = [20,-15]
A = [2 -3 5; -1 2 -9]

In [ ]:
pinv(A)*y

In [ ]:
eye(size(A)[2]) - (pinv(A)*A)

In [ ]:
x = pinv(A)*y + (eye(size(A)[2]) - pinv(A)*A)*[0,0,0]

In [ ]:
A*x

In [ ]:
([0,0,0]-pinv(A)*y)

In [ ]:
(eye(size(A)[2]) - pinv(A)*A)

In [ ]:
([0,0,0]-pinv(A)*y)\(eye(size(A)[2]) - pinv(A)*A)

In [ ]:
[-2.70309e-16,  2.47783e-16,  -2.47783e-16]

In [ ]:
x = pinv(A)*y + (eye(size(A)[2]) - pinv(A)*A)*[-2.70309e-16,2.47783e-16,-2.47783e-16]

In [ ]:
A*x

In [ ]:
([1,1,1]-pinv(A)*y)\(eye(size(A)[2]) - pinv(A)*A)

In [ ]:
x = pinv(A)*y + (eye(size(A)[2]) - pinv(A)*A)*[0.0406416,0.0310789,0.00239068]

In [ ]:
A*x

In [ ]:
y\A

In [2]:
###
### Planteamiento del problema como red neuronal
###

# unknowns
n = 3;

# equations
m = 2;

# valid values of the unknowns
xs = rand(n);

# weights of a neural net layer
W  = rand(Uniform(-sqrt(n+1),sqrt(n+1)), (n+1,m));

# real estimation of xs flowing through the layer
ys = map(σ, W'*[1.0;xs]);
;
# Conociendo W y ys, encontrar xs que cumplan ys = map(σ, W'*[1.0;xs])

In [3]:
###
### Planteamiento del problema como álgebra lineal
###

A = W'[:,2:end];
b = ys-W'[:,1]
;
# Encontrar z tal que Az = b

In [4]:
A

2x3 Array{Float64,2}:
 -0.968031   1.96318   -1.04327 
  1.6666    -0.190253   0.818914

In [5]:
b

2-element Array{Float64,1}:
  1.02539 
 -0.189963

In [37]:
pinv(A)

3x2 Array{Float64,2}:
  0.10972   0.567773
  0.507469  0.359189
 -0.105399  0.149082

In [35]:
x = pinv(A)*b + (eye(size(A)[2]) - pinv(A)*A)*[4,0,0]

3-element Array{Float64,1}:
  0.644495 
  0.0226016
 -1.53835  

In [36]:
abs(A*x-b.-0) .< 1e-5

2-element BitArray{1}:
 true
 true

In [9]:
eye(size(A)[2]) - pinv(A)*A

3x3 Array{Float64,2}:
  0.159961  -0.10738    -0.35049 
 -0.10738    0.0720831   0.23528 
 -0.35049    0.23528     0.767956

In [23]:
rref([A b])

2x4 Array{Float64,2}:
 1.0  0.0   0.456393  -0.0575992
 0.0  1.0  -0.306372   0.49391  

In [22]:
function rref{T}(A::Matrix{T})
    nr, nc = size(A)
    U = copy!(similar(A, T <: Complex ? Complex128 : Float64), A)
    e = eps(norm(U,Inf))
    i = j = 1
    while i <= nr && j <= nc
        (m, mi) = findmax(abs(U[i:nr,j]))
        mi = mi+i - 1
        if m <= e
            U[i:nr,j] = 0
            j += 1
        else
            for k=j:nc
                U[i, k], U[mi, k] = U[mi, k], U[i, k]
            end
            d = U[i,j]
            for k = j:nc
                U[i,k] /= d
            end
            for k = 1:nr
                if k != i
                    d = U[k,j]
                    for l = j:nc
                        U[k,l] -= d*U[i,l]
                    end
                end
            end
            i += 1
            j += 1
        end
    end
    U
end

rref (generic function with 1 method)

In [34]:
let FGH = svd(A),
    F = FGH[1],
    G = FGH[2],
    H = FGH[3]
    
    println(F)
    println()
    println(G)
    println()
    println(H)
end

[-0.8331924184620049 0.5529831767942276
 0.5529831767942275 0.8331924184620045]

[2.7866176129663307,1.2644041761461542]

[0.6201638826306439 0.6748596893819094
 -0.6247414564758897 0.733222364931946
 0.47444164155443747 0.08336284070829612]
